In [7]:
from scipy.optimize import minimize
import numpy as np

In [2]:
def get_nu_star(xi, params):

    def objective(nu):

        if xi(nu, **params)>0:
            return -(np.log(xi(nu, **params)) - 0.5*np.dot(nu.T, nu))
        else:
            return 1e80
        
    nu0 = np.zeros(params["d"])

    result = minimize(objective, nu0, method='SLSQP')

    if result.success:
        return result.x/np.linalg.norm(result.x)
    else:
        raise ValueError("L'optimisation n'a pas réussi à converger.")

In [4]:
def asian_function(y, S0, v, mu, maturity, strike, rate, d):

    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*T)/d + v*np.sqrt(T/d)*y_cum))/d - strike, 0)

    return payoff

In [5]:
d = 16
T = 1
r = 0.05
S0 = 50
K = 45
v = 0.1

params_asian = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r
}

In [8]:
nu = get_nu_star(asian_function, params_asian)
nu

array([0.40982835, 0.38528402, 0.36056971, 0.33569056, 0.31065248,
       0.2854611 , 0.26012196, 0.23464137, 0.20902524, 0.18327985,
       0.15741169, 0.13142738, 0.10533349, 0.07913696, 0.05284445,
       0.02646301])

In [9]:
def knock_out_function(y, S0, v, mu, maturity, strike, rate, barrier, d):

    d = len(y)
    k = np.arange(1, d + 1)
    y_cum = np.cumsum(y)

    s = S0*np.exp(mu*maturity + v*np.sqrt(maturity/d)*y_cum[-1])
    payoff = np.exp(-rate*maturity)*max(S0*np.sum(np.exp(mu*(k*maturity)/d + v*np.sqrt(maturity/d)*y_cum))/d - strike, 0)*(1 if s <= barrier else 0)

    return payoff

In [10]:
d = 16
T = 1
r = 0.05
S0 = 50
K = 50
B = 60
v = 0.1

params_barrier = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "maturity": T,
    "strike": K,
    "rate": r, 
    "barrier": B
}

In [14]:
nu = get_nu_star(knock_out_function, params_barrier)
nu

array([0.40812684, 0.38420128, 0.3600129 , 0.33557173, 0.31088697,
       0.28597045, 0.26083241, 0.23548451, 0.20993855, 0.18420724,
       0.15830259, 0.13223834, 0.10602771, 0.07968461, 0.05322245,
       0.02665596])

In [15]:
def basket_options_function(y, S0, v, mu, sq_correl, maturity, strike, alpha, rate, d):

    y_tilde = np.dot(sq_correl, y)
    payoff = np.exp(-rate*maturity)*max(np.sum(alpha*S0*np.exp(mu*T + v*np.sqrt(maturity)*y_tilde)) - strike, 0)

    return payoff

In [16]:
d = 40
T = 1
r = 0.05
S0 = np.random.uniform(20, 80, d)
K = 45
alpha = np.ones(d)/d

c = 0.1
sigma = c*np.ones((d, d)) + (1-c)*np.eye(d)
v = np.linspace(0.1, 0.4, d)

params_basket = {
    "d": d,
    "S0": S0,
    "v": v,
    "mu": r-0.5*v**2,
    "sq_correl": np.linalg.cholesky(sigma),
    "maturity": T,
    "strike": K,
    "alpha": alpha,
    "rate": r
}

In [18]:
nu = get_nu_star(basket_options_function, params_basket)
nu

array([0.31098507, 0.28295302, 0.23645495, 0.23223689, 0.22843953,
       0.21541919, 0.1935192 , 0.18289986, 0.17008414, 0.15646833,
       0.18028326, 0.13838783, 0.16428681, 0.17851825, 0.17006082,
       0.10616331, 0.11058391, 0.1305841 , 0.11924611, 0.11221127,
       0.14567904, 0.14512865, 0.09592657, 0.11956511, 0.08323065,
       0.13219729, 0.14435168, 0.14742942, 0.07028179, 0.14406743,
       0.0917284 , 0.05783134, 0.11364253, 0.1335936 , 0.0724341 ,
       0.09489487, 0.09261313, 0.13534261, 0.12829844, 0.15843373])

## Test fonction